In [1]:
import enum

import isort
import sys
import platform
import importlib
from types import ModuleType

In [2]:
# module_names = isort.stdlibs.py38.stdlib

In [3]:
def get_stdlib_packages():
    if sys.version_info.minor == 10:
        module_names = sys.stdlib_module_names
    else:
        module_names = isort.stdlibs.py38.stdlib

    external_packages = set()
    for name in module_names:
        if name[0] == '_' or name == 'this' or name == 'antigravity':
            continue
        external_packages.add(name)
    return external_packages

In [4]:
def task1():
    external_packages = get_stdlib_packages()
    v = sys.version_info
    print(f'Python {v.major}.{v.minor}.{v.micro} on {platform.system()} {platform.release()}')
    print(f'Stdlib contains {len(external_packages)} external modules and packages: ')
    print(external_packages)

In [5]:
task1()

Python 3.8.10 on Linux 5.15.0-50-generic
Stdlib contains 212 external modules and packages: 
{'curses', 'traceback', 'concurrent', 'cgi', 'asynchat', 'hmac', 'subprocess', 'resource', 'codecs', 'lzma', 'zipapp', 'bz2', 'nntplib', 'telnetlib', 'argparse', 'ipaddress', 'pathlib', 'reprlib', 'secrets', 'wsgiref', 'base64', 'urllib', 'runpy', 'fnmatch', 'uu', 'uuid', 'cmath', 'mailcap', 'pickletools', 'sre_compile', 'pyclbr', 'imaplib', 'formatter', 'timeit', 'collections', 'fileinput', 'importlib', 'sqlite3', 'string', 'locale', 'textwrap', 'zlib', 'struct', 'tarfile', 'zipimport', 'mmap', 'parser', 'cmd', 'time', 'cProfile', 'io', 'json', 'enum', 'dummy_threading', 'nis', 'errno', 'bisect', 'statistics', 'weakref', 'platform', 'select', 'gc', 'ossaudiodev', 'sre_constants', 'operator', 'shutil', 'msvcrt', 'faulthandler', 'rlcompleter', 'asyncio', 'email', 'csv', 'atexit', 'itertools', 'logging', 'marshal', 'types', 'abc', 'copyreg', 'winsound', 'queue', 'sys', 'copy', 'functools', 'array

In [6]:
def get_real_packages(package_names):
    real_modules = set()
    not_importable_modules = set()
    for name in package_names:
        try:
            importlib.import_module(name)
            real_modules.add(name)
            # if name != 'importlib':
            #     del sys.modules[name]
        except:
            not_importable_modules.add(name)
    return real_modules, not_importable_modules

In [7]:
def get_real():
    external_packages = get_stdlib_packages()
    return get_real_packages(external_packages)

In [8]:
def task2():
    v = sys.version_info
    real_modules, not_importable_modules = get_real()
    print(f'These StdLib packages on Python-{v.major}.{v.minor}.{v.micro}/{platform.system()} {platform.release()} '
          f'are not importable:')
    print(not_importable_modules)

    # for name in real_modules:
    #     del sys.modules[name]

In [9]:
task2()

These StdLib packages on Python-3.8.10/Linux 5.15.0-50-generic are not importable:
{'tkinter', 'msilib', 'msvcrt', 'winsound', 'turtledemo', 'ensurepip', 'winreg', 'sre', 'turtle'}


In [10]:
def module_dependency(module_names, name):
    if name not in module_names:
        raise Exception(f'{name} is not importable module')
    dp_names = set()

    try:
        importlib.import_module(name)
    except:
        print(f'err name: {name}')
    for key, val in vars(sys.modules[name]).items():
        if isinstance(val, ModuleType):
            md_name = val.__name__

            try:
                index = md_name.index(".")
                md_name = md_name[0:index]
            except:
                pass

            dp_names.add(md_name)
            # print(f'key: {key}, type: {type(val)}, val: {md_name}, module: {md_name}')

    # del sys.modules[name]

    return dp_names

In [11]:
def core_modules(real_modules):
    # real_modules, _ = get_real()
    core_module_names = set()
    for r_name in real_modules:
        dp_names = module_dependency(real_modules, r_name)
        if len(dp_names) == 0:
            core_module_names.add(r_name)

    return core_module_names


In [12]:
def most_dependent_modules(real_modules):
    dp_dict = dict((name, 0) for name in real_modules)
    for md_name in real_modules:
        count = len(module_dependency(real_modules, md_name))
        dp_dict[md_name] = count

    sorted_dp_dict = dict(sorted(dp_dict.items(), key=lambda item: item[1], reverse=True))
    i = 0
    most_dependent_module_names = dict()
    for k, v in sorted_dp_dict.items():
        if i > 5:
            break
        most_dependent_module_names[k] = v
        i = i + 1

    return most_dependent_module_names



In [13]:
def task3():
    real_modules, not_importable_modules = get_real()

    # re-import libs:
    for r in real_modules:
        pass

    # print(f'real modules: {real_modules}')
    core_module_names = core_modules(real_modules)

    dp_dict = most_dependent_modules(real_modules)

    print(f'The following StdLib packages are most dependent:')
    for k, v in dp_dict.items():
        print(f'{k}: {v}')
    print(f'The {len(core_module_names)} core packages are:')
    print(core_module_names)

    # for name in real_modules:
    #     del sys.modules[name]

In [14]:
task3()

The following StdLib packages are most dependent:
zipfile: 17
inspect: 16
pdb: 15
pydoc: 14
subprocess: 13
logging: 12
The 60 core packages are:
{'quopri', 'errno', 'bisect', 'posix', 'builtins', 'imghdr', 'unicodedata', 'select', 'gc', 'ossaudiodev', 'sre_constants', 'readline', 'heapq', 'sre_parse', 'resource', 'termios', 'operator', 'colorsys', 'chunk', 'syslog', 'faulthandler', 'itertools', 'atexit', 'grp', 'wsgiref', 'marshal', 'math', 'types', 'numbers', 'abc', 'copyreg', 'audioop', 'cmath', 'stat', 'spwd', 'sys', 'copy', 'functools', 'decimal', 'difflib', 'array', 'stringprep', 'symbol', 'test', 'token', 'pwd', 'zlib', 'keyword', 'struct', 'tty', 'ast', 'mmap', 'contextvars', 'parser', 'fcntl', 'time', 'binascii', 'sndhdr', 'lib2to3', 'nis'}


In [15]:
def explore_package(package_name):
    importlib.import_module(package_name)
    pack = sys.modules[package_name]
    file_name = 'builtin_binary'
    try:
        file_name = pack.__file__
        n = len(file_name)
        # print(file_name[n-3:n])
        if file_name[n-3:n] == '.py':
            # print(file_name)
            return package_name, file_name, 0
        elif file_name[n-3:n] == '.so':
            # print(file_name)
            return package_name, file_name, 1
        else:
            raise Exception(f'unsupported file: {file_name}')
    except:
        return package_name, file_name, 2

p, f, r = explore_package('locale')
print(r)

0


In [16]:
real_modules, _ = get_real()

for md in real_modules:
    p, f, r = explore_package(md)
    print(f'package name: {p}, file: {f}, type: {r}')

package name: curses, file: /usr/lib/python3.8/curses/__init__.py, type: 0
package name: traceback, file: /usr/lib/python3.8/traceback.py, type: 0
package name: cgi, file: /usr/lib/python3.8/cgi.py, type: 0
package name: concurrent, file: /usr/lib/python3.8/concurrent/__init__.py, type: 0
package name: asynchat, file: /usr/lib/python3.8/asynchat.py, type: 0
package name: hmac, file: /usr/lib/python3.8/hmac.py, type: 0
package name: subprocess, file: /usr/lib/python3.8/subprocess.py, type: 0
package name: resource, file: /usr/lib/python3.8/lib-dynload/resource.cpython-38-x86_64-linux-gnu.so, type: 1
package name: codecs, file: /usr/lib/python3.8/codecs.py, type: 0
package name: lzma, file: /usr/lib/python3.8/lzma.py, type: 0
package name: zipapp, file: /usr/lib/python3.8/zipapp.py, type: 0
package name: bz2, file: /usr/lib/python3.8/bz2.py, type: 0
package name: nntplib, file: /usr/lib/python3.8/nntplib.py, type: 0
package name: telnetlib, file: /usr/lib/python3.8/telnetlib.py, type: 0


In [27]:
import inspect
import fileinput
import socket
# from types import
import enum

for name, obj in inspect.getmembers(socket):
        if inspect.isclass(obj):
            if issubclass(obj, enum.Enum):
                print(f'{obj}, enum')
            else:
                print(f'{obj}, type: {type(obj).__class__}')

<enum 'AddressFamily'>, enum
<enum 'AddressInfo'>, enum
<enum 'IntEnum'>, enum
<enum 'IntFlag'>, enum
<enum 'MsgFlag'>, enum
<class 'socket.SocketIO'>, type: <class 'type'>
<enum 'SocketKind'>, enum
<class '_socket.socket'>, type: <class 'type'>
<class 'socket._GiveupOnSendfile'>, type: <class 'type'>
<class '_socket.socket'>, type: <class 'type'>
<class 'OSError'>, type: <class 'type'>
<class 'socket.gaierror'>, type: <class 'type'>
<class 'socket.herror'>, type: <class 'type'>
<class 'socket.socket'>, type: <class 'type'>
<class 'socket.timeout'>, type: <class 'type'>


In [31]:
real_modules, _ = get_real()

md_class_count = dict()

for md in real_modules:
    count = 0
    # cl = set()
    for name, obj in inspect.getmembers(sys.modules[md]):
        if inspect.isclass(obj):
            if issubclass(obj, enum.Enum):
                # print(f'{obj}, enum')
                continue
            else:
                # print(f'{obj}, type: {type(obj).__class__}')
                count = count + 1
                # cl.add(obj.__name__)
    md_class_count[md] = count

sorted_cl_dict = dict(sorted(md_class_count.items(), key=lambda item: item[1], reverse=True))
i = 0
largest_pack_cl = dict()
for k, v in sorted_cl_dict.items():
    if i > 5:
        break
    largest_pack_cl[k] = v
    i = i + 1

print(largest_pack_cl)




{'ast': 122, 'builtins': 93, 'asyncio': 45, 'ctypes': 43, 'typing': 33, 'argparse': 26}
